## Food2Fork API ##
www.food2fork.com/about/api

In [1]:
import re
import requests
import urllib2
from urllib import quote
import json
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
import nltk

In [2]:
%matplotlib inline

### API credentials ###

In [3]:
def loadCredentials():
    filename = 'secrets.txt'
    for line in open(filename).readlines():
        if "API" in line:    
            api_key = line.split(": ")[1].translate(None,'\n')
        
    return api_key

In [4]:
# Load the API Key
API_KEY = loadCredentials()

# Food2Fork links
_URL_API = 'http://food2fork.com/api/'
_URL_SEARCH = _URL_API + 'search?' + "key={0}".format(API_KEY)
_URL_GET = _URL_API + 'get?' + "key={0}".format(API_KEY)
_HEADER = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}

### Search the Food2Fork API ###

In [5]:
def search(query, page=1):   
    """Return a list of recipes from the Food2Fork.com database"""

    # Format the request URL    
    api_request = _URL_SEARCH+"&q={query}&page={page}".format(query=query,page=page)

    # Make the request
    request  = urllib2.Request(api_request, headers=_HEADER)
    response = urllib2.urlopen(request)
    raw = response.read()
    json_obj = json.loads(raw)  
    
    return json_obj['recipes']       

In [6]:
# Search for chicken salad recipes
recipes = search('Chicken salad')
print(recipes[0]['image_url'])
recipes[0]

http://static.food2fork.com/Bacon2BWrapped2BJalapeno2BPopper2BStuffed2BChicken2B5002B5909939b0e65.jpg


{u'f2f_url': u'http://food2fork.com/view/35120',
 u'image_url': u'http://static.food2fork.com/Bacon2BWrapped2BJalapeno2BPopper2BStuffed2BChicken2B5002B5909939b0e65.jpg',
 u'publisher': u'Closet Cooking',
 u'publisher_url': u'http://closetcooking.com',
 u'recipe_id': u'35120',
 u'social_rank': 100.0,
 u'source_url': u'http://www.closetcooking.com/2012/11/bacon-wrapped-jalapeno-popper-stuffed.html',
 u'title': u'Bacon Wrapped Jalapeno Popper Stuffed Chicken'}

### Request a specific recipe from the API ###

In [7]:
def getRecipe(recipe_id):
    """Return the recipe specified by *recipe_id* from Food2Fork's database"""

    # Format the request URL
    api_request = _URL_GET+"&rId={ID}".format(ID=recipe_id)
    
    # Make the request
    request  = urllib2.Request(api_request, headers=_HEADER)
    response = urllib2.urlopen(request)
    raw = response.read()
    json_obj = json.loads(raw)  
    
    return json_obj['recipe']

In [8]:
r = getRecipe('35130')

### Scrape ingredients from a Food2Fork.com page ###

In [9]:
def getIngredients(json_obj):
    r = getRecipe(json_obj['recipe_id'])
    return [i.strip('\n') for i in r['ingredients']]

In [10]:
getIngredients(recipes[0])

[u'4 small chicken breasts, pounded thin',
 u'salt and pepper to taste',
 u'4 jalapenos, diced',
 u'4 ounces cream cheese, room temperature',
 u'1 cup cheddar cheese, shredded',
 u'8 slices bacon']

### Recipe Class ###

In [12]:
class Recipe(object):
    
    def __init__(self,json_obj):
        self._title = json_obj['title']
        self._image_url = json_obj['image_url']
        self._publisher = json_obj['publisher']
        self._publisher_url = json_obj['publisher_url']
        self._recipe_id = json_obj['recipe_id']
        self._social_rank = json_obj['social_rank']
        self._source_url = json_obj['source_url']        
        self._ingredients = getIngredients(json_obj)
                            
    def __repr__(self):
        return self._title
    
    def __str__(self):
        return self._title
    
    @property
    def title(self):
        return self._title
    
    @property
    def ingredients(self):
        return self._ingredients
    
    @property
    def publisher(self):
        return self._publisher
    
    @property
    def URL(self):
        return self._source_url

In [13]:
BaconChicken = Recipe(recipes[0])
print(BaconChicken)
BaconChicken.ingredients

Bacon Wrapped Jalapeno Popper Stuffed Chicken


[u'4 small chicken breasts, pounded thin',
 u'salt and pepper to taste',
 u'4 jalapenos, diced',
 u'4 ounces cream cheese, room temperature',
 u'1 cup cheddar cheese, shredded',
 u'8 slices bacon']